## Installing Required Libraries

In [1]:
!pip install openai chromadb sentence_transformers -q

In [3]:
!pip install langchain unstructured -q 

## Load Documents

In [4]:
from langchain.document_loaders import DirectoryLoader
directory = 'countries'

def load_doc(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_doc(directory)
len(documents)

5

## Splitting to Chunks

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_doc(documents,chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_doc(documents)
print(len(docs))

23


In [8]:
docs[0].page_content

"Let's explore the culture, festivals, and history of England in detail.\n\n\n\n\n\nEngland's Culture:**\n\n**1. Cultural Heritage:** England is renowned for its rich cultural heritage, which has significantly influenced the world. Its contributions include literature, theater, music, and art. Notable figures include William Shakespeare, Charles Dickens, and The Beatles.\n\n**2. Language:** English, the official language, has become a global lingua franca. England's language and literature have played a pivotal role in shaping the modern world.\n\n**3. Arts and Entertainment:** England is home to world-class theaters like London's West End, where numerous plays and musicals are performed. The country has a strong tradition of classical music and hosts events like the BBC Proms.\n\n**4. Cuisine:** English cuisine includes classics like fish and chips, roast dinners, and afternoon tea. The country also has a diverse culinary scene influenced by immigration, with a variety of internationa

In [9]:
docs[22].page_content

"Pakistan's culture, festivals, and history are deeply rooted in its Islamic heritage and diverse cultural influences. The country's rich traditions and historical legacy contribute to its unique identity."

## Embedding 

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


## Vector Database

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)